In [136]:
import sys
sys.path.append('I:/YaoTony/code/x8313')
sys.path.append('I:/YaoTony/saleem tasks')
import Harville
#ys.path.append('/Users/saleemhuda/Dropbox/X8313_Live')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd
from horse.betsim import data
import os.path
track_detail = os.path.join(data.__path__._path[0], 'track_detail.csv')
df_track_detail = pd.read_csv(track_detail, index_col=['x8_track_sym'])
map_jcp_itsp = df_track_detail.set_index('jcp_track_sym')['itsp_track_sym'].to_dict()
map_rebates_win = df_track_detail.WN.to_dict()
df_hist = pd.read_csv('I:/YaoTony/saleem tasks/dfX_hist.csv')#i did not find dfX_hist in local database, so I use the one in dropbox
df_hist['payout_win'] = df_hist['final_tote_odds']+1
df_hist['rebate_win'] = df_hist['track_id'].map(lambda x:map_rebates_win.get(x,0.0))

In [137]:
#set nan rebate to 0
import numpy as np
for key in map_rebates_win.keys():
    if not np.isfinite(map_rebates_win[key]):
        map_rebates_win[key]=0

In [138]:
from horse.betsim.math import compute_probs_from_odds
from horse.betsim.wrap.jcapper import JCapper
# load JCapper Breeders Cup Results file
jcp = JCapper(verbose=True)
jcp.load(date_range(date(2017, 7, 3), date(2017, 7, 9)))#you can adjust the date
jcp.add_computed_columns()

print('load_jcapper: loaded %d rows' % (len(jcp.df)))

dfpay = jcp.df_payout.copy() #exotic bets payouts in easier form
dfres = jcp.df.copy() #race results dataframe

jcapper.load(2017-07-03)
jcapper.load(2017-07-04)
jcapper.load(2017-07-05)
jcapper.load(2017-07-06)
jcapper.load(2017-07-07)
jcapper.load(2017-07-08)
jcapper.load(2017-07-09)
JCapper.load() filtering scratched horses reduced from 9447 to 8500


I:\YaoTony\code\x8313\horse\betsim\wrap\jcapper.py:206: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_payout = concat([df_payout, wps])
I:\YaoTony\code\x8313\horse\betsim\wrap\jcapper.py:273: RuntimeWarning: invalid value encountered in log
  self.df['log_ratio_effectivestarters_morningline'] = -1.0 * log(self.df.num_effective_starters_morning_line / self.df.num_starters)


load_jcapper: loaded 8500 rows


In [139]:
#only keep trifecta payout data
dfpay_tri = dfpay[dfpay['wager_name'].isin(['Triactor','Trifecta'])].reset_index(drop=True)
#build a function that split one column in df to 3
splitdf = lambda x: pd.Series([i for i in x.split('-')])
splitdf1 = lambda x: pd.Series([i for i in x.split('_')])
dfpay_tri_place = dfpay_tri['wager_winningPGM'].apply(splitdf)
dfpay_tri_place.columns = ['first_place','second_place','third_place']
dfpay_tri = pd.concat([dfpay_tri, dfpay_tri_place], axis=1)
dfpay_tri_track = dfpay_tri['race_id'].apply(splitdf1)
dfpay_tri['track_id'] = dfpay_tri_track[0]
dfpay_tri['payout_1dollar']=dfpay_tri['wager_payout']/dfpay_tri['wager_unit']

In [140]:
#only keep the races that are also in df_hist
dfpay_tri['use'] = dfpay_tri['race_id'].isin(list(df_hist['race_id']))
dfpay_tri_use = dfpay_tri[dfpay_tri['use']==True]


In [141]:
track_set = set(dfpay_tri_use['track_id'])
len(track_set)

40

In [142]:
dfpay_tri_use.head()

,race_id,wager_number,wager_numright,wager_payout,wager_pool,wager_type,wager_unit,wager_winningPGM,wager_name,wager_axis,first_place,second_place,third_place,track_id,payout_1dollar,use
0,FEX_20170704_1,1,NaN,69.5,18856.0,A,2.0,1-5-4,Triactor,None,1,5,4,FEX,34.75,True
1,FEX_20170704_2,1,NaN,132.9,8562.0,A,2.0,5-1-2,Triactor,None,5,1,2,FEX,66.45,True
2,ASD_20170705_1,1,NaN,42.1,1809.0,A,2.0,2-1-3,Triactor,None,2,1,3,ASD,21.05,True
3,ASD_20170705_2,1,NaN,6.8,2559.0,A,2.0,4-1-2,Triactor,None,4,1,2,ASD,3.40,True
4,ASD_20170707_1,1,NaN,15.6,2057.0,A,2.0,2-5-3,Triactor,None,2,5,3,ASD,7.80,True


In [143]:
def compute_advantage_new(df,track):
    pct_win = len(df[df['is_paid']>1])/len(df)
    pct_loss = 1.0 - pct_win
    mean_odds = df[df.is_paid>0]['payout_1dollar'].mean()-1
    advantage = pct_win - pct_loss / (mean_odds+map_rebates_win[track])
    #print(advantage)
    return advantage

In [144]:
advan = {}
dfpay_tri_use['is_paid'] = 0
#Here we suppose alpha and beta are equal to 1 for every track, since alpha and beta may change depending on different historical set. 
#But we can optimize it through some codes.
#Para_Harville = {}

for track in track_set:
    #Para_Harville[track] = [1, 1]#the first number is the optimal alpha in Harville, second is the optimal beta
    dfpay_track = dfpay_tri_use[dfpay_tri_use['track_id']==track]
    dfpay_track=dfpay_track.reset_index(drop=True)
    raceset = set(dfpay_track['race_id'])
    for race in raceset:
        df_sub = df_hist[df_hist['race_id']==race]
        dfpay_race = dfpay_track[dfpay_track['race_id']==race]
        # here we set is_paid = 0  for races who has less than 3 runners. We may try better method to deal with this later
        if len(df_sub)>2:
        #fav is prob ranking 1 of trifecta combination and its prob
            fav = Harville.harville_multiprob(list(df_sub['runner_program_number_pp']),list(df_sub['prob_x8runner_HDWPSRRating_norm_par']),'Trifecta')[0]
            if (dfpay_race['first_place']==fav[0]).all() and (dfpay_race['second_place']==fav[1]).all() and (dfpay_race['third_place']==fav[2]).all():
                dfpay_track.loc[dfpay_track['race_id']==race,'is_paid'] = 1
    advan[track] = compute_advantage_new(dfpay_track,track)#nan is when 'is_paid' is all 0 in the track
    

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [145]:
advan #nan is when 'is_paid' is all 0 in the track

{'ALB': -0.02710027100271003,
 'APX': nan,
 'ARP': nan,
 'ASD': -0.07117437722419928,
 'BEL': nan,
 'BTP': nan,
 'CBY': -0.1149425287356322,
 'CTX': -0.04739336492890995,
 'DEL': -0.0223463687150838,
 'ELP': -0.014513788098693758,
 'EMD': -0.028915662650602407,
 'EVD': nan,
 'FEX': -0.05128205128205128,
 'FLX': nan,
 'FMT': -0.04366812227074236,
 'FPK': -0.03289473684210526,
 'GPR': nan,
 'GPX': nan,
 'GRP': -0.059880239520958084,
 'HPO': nan,
 'HST': nan,
 'IND': nan,
 'LAD': nan,
 'LAX': nan,
 'LRC': -0.04519774011299436,
 'LRL': nan,
 'LSX': -0.043859649122807015,
 'MDX': nan,
 'MTH': nan,
 'NPX': nan,
 'PEN': -0.05509641873278237,
 'PHA': nan,
 'PID': -0.037037037037037035,
 'PLN': -0.11235955056179775,
 'PRM': -0.06968641114982578,
 'RUI': nan,
 'SAX': nan,
 'SUF': -0.08097165991902833,
 'TDN': -0.033955857385398976,
 'WYO': nan}